In [ ]:
# unified_abcc_chicago_epi_plot.R
# Single script to plot ABCC/CHiC interactions together with epigenetic tracks
# Requires: data.table, Sushi
# Assumes input files similar to user's examples. Adjust paths and object names as needed.

library(data.table)
library(Sushi)    # for plotBed, plotBedpe, plotGenes, labelgenome

library(ggplot2)
library(GenomicRanges)

library(biomaRt)


In [2]:
atac_file      <- "/rds/general/user/malyshev/home/miniPCHiC/hILCs/spivakov_pchic_ILC_CD4/data/peaks/ATAC/ILC3_ATAC_peaks.bed"        # replace with your file/track
k27ac_file     <- "/rds/general/user/malyshev/home/miniPCHiC/hILCs/spivakov_pchic_ILC_CD4/data/peaks/CHIP/ILC3_H3K27ac_peaks.bed"
k4me3_file     <- "/rds/general/user/malyshev/home/miniPCHiC/hILCs/spivakov_pchic_ILC_CD4/data/peaks/CHIP/ILC3_H3K4me3_peaks.bed"

epiread <- function(epifile){
    epiprofread <- fread(epifile)
    setnames(epiprofread, colnames(epiprofread[,1:3]), c("chr", "peak_start", "peak_end"))
    epiprofread[,chr := gsub("chr", "", chr)]
    return(epiprofread)
}

k27ac <- epiread(k27ac_file)
atac <- epiread(atac_file)
k4me3 <- epiread(k4me3_file)

In [37]:
## plotting gene tracks

gene_track_plot <- function(gene_name = gene_name, chrom = chrom, bmap = bmap, bigStart = bigStart, bigEnd = bigEnd, midpoint = midpoint){
    
    mart <- useEnsembl(
    biomart = "ensembl", 
    dataset = "hsapiens_gene_ensembl"      
    )

    gene_annotation <- getBM(
      attributes = c("chromosome_name", "exon_chrom_start", "exon_chrom_end", "external_gene_name", "strand", "gene_biotype"),
      filters = c("chromosome_name", "start", "end"),
      values = list(chrom, bigStart*1000, bigEnd*1000),
      mart = mart
    )

    gene_annotation <- as.data.table(gene_annotation)

    gene_annotation[, `:=`(
      chromosome_name = as.character(chromosome_name),
      strand = as.character(strand)
    )]
    
    # keep only protein-coding genes
    gene_annotation <- gene_annotation[gene_biotype == "protein_coding"]

    gene_annotation <- gene_annotation[, .(
      chrom = chromosome_name, 
      start = exon_chrom_start, 
      stop = exon_chrom_end, 
      gene = external_gene_name, 
      strand, 
      colour = adjustcolor("grey23", alpha.f = 0.5)
      #colour = ifelse(strand == 1, "darkgoldenrod", "dodgerblue4")
      )]

    for(current_gannot in gene_annotation[, sort(unique(gene))]){
          if(grepl(current_gannot, gene_name) == TRUE){
            gene_annotation[(gene == current_gannot), colour := "dodgerblue4"]
          }
        }

    gene_annotation[(strand == 1 | strand == -1), strand := ifelse(strand == 1, "+", "-")]
    gene_annotation <- gene_annotation[gene!=""]
        
    return(gene_annotation)

}
   

In [54]:
library(ggplot2)
library(dplyr)
library(patchwork)

batch_epi_plot_short <- function(peakm, imputed, bait_list, bmap, rmap,
                                 window_left = NULL, window_right = NULL, zoom = TRUE) {
  
  for (id in bait_list) {
    message("Processing bait: ", id)
    gene_name <- bmap[V4 == id]$V5[1]
    
    # ----------------- Zoom/window definition -----------------
    if (zoom) {
      if (is.null(window_left)) {
        window_left <- (min(as.numeric(peakm[baitID == id][baitChr == oeChr]$dist))) / 1000 - 20
      }
      if (is.null(window_right)) {
        window_right <- (max(as.numeric(peakm[baitID == id][baitChr == oeChr]$dist))) / 1000 + 20
      }
    } else {
      if (is.null(window_left)) {
        window_left <- (min(as.numeric(peakm[baitID == id][baitChr == oeChr]$dist))) / 1000 - 20
      }
      if (is.null(window_right)) {
        window_right <- (max(as.numeric(peakm[baitID == id][baitChr == oeChr]$dist))) / 1000 + 20
      }
    }
    
    midpoint <- (bmap[V4==id]$V2 + bmap[V4==id]$V3) / 2000  # kb scale
    bigStart <- midpoint - abs(window_left)
    bigEnd   <- midpoint + abs(window_left)
    chrom    <- bmap[V4==id][1,]$V1
    
    peakm_sel <- peakm[baitChr == chrom][((bait_start_fres > bigStart*1000) & (oe_end_fres < bigEnd*1000)) | 
                                           ((oe_start_fres > bigStart*1000) & (bait_end_fres < bigEnd*1000))]
    
    # ----------------- Output file -----------------
    fn <- paste0("~/miniPCHiC/hILCs/hILCs_run2/abc_results/", 
                 id, "_", window_left, "kb_left_", window_right, "kb_right_zoom_", zoom, "_5kb.pdf")
    
    pdf(fn, width = 10, height = 10)
    
    # ----------------- Contact plot -----------------
    this <- imputed[baitID == id]
    
    this_abcc    <- unique(xpm[baitID_5kb == id & !is.na(ABC.Score)], by = c("baitID_5kb", "oeID_5kb"))
    this_chicago <- unique(xpm[baitID_5kb == id & !is.na(chicago_score_5kb)], by = c("baitID_5kb", "oeID_5kb"))
    this_both    <- merge(this_abcc, this_chicago, by = c("baitID_5kb", "oeID_5kb"))
    
    this <- this %>%
      mutate(group = case_when(
        otherEndID %in% this_both$oeID_5kb ~ "Both",
        otherEndID %in% this_abcc$oeID_5kb ~ "ABCC",
        otherEndID %in% this_chicago$oeID_5kb ~ "CHiCAGO",
        TRUE ~ "None"
      ))
    
    color_map <- c("ABCC" = "blue", "CHiCAGO" = "red", "Both" = "green", "None" = "black")
    
    rmap_abcc    <- rmap[V4 %in% this_abcc$oeID_5kb]
    rmap_chicago <- rmap[V4 %in% this_chicago$oeID_5kb]
    rmap_both    <- rmap[V4 %in% this_both$oeID_5kb]
    
    highlights <- bind_rows(
      mutate(as.data.frame(rmap_abcc), group = "ABCC", col = "blue"),
      mutate(as.data.frame(rmap_chicago), group = "CHiCAGO", col = "red"),
      mutate(as.data.frame(rmap_both), group = "Both", col = "green")
    )
    
    g_contacts <- ggplot(this, aes(x = (oe_start + oe_end)/2)) +
          geom_line(aes(y = N_imp.x), linetype = "dashed") +
          geom_point(
            aes(
              y = N_obs,
              colour = group,                # border colour
              fill   = ifelse(N_obs >= N_imp.x, group, NA)  # fill same as group if above expected
            ),
            shape = 21, size = 2.2, stroke = 0.7
          ) +
          scale_colour_manual(values = color_map, guide = "none") +
          scale_fill_manual(values = color_map, na.value = "white", guide = "none") +
          coord_cartesian(xlim = c(bigStart * 1000, bigEnd * 1000)) +
          theme_minimal(base_size = 10) +
          theme(
            panel.grid = element_blank(),
            panel.border = element_blank(),
            axis.text = element_text(size = 10),
              axis.line    = element_line(colour = "black")
          ) +
          labs(y = "Contact", x = NULL) +
          geom_rect(
            data = highlights,
            aes(xmin = V2, xmax = V3),
            fill = highlights$col,
            ymin = -Inf, ymax = Inf, alpha = 0.1, inherit.aes = FALSE
          )

    
    # ----------------- Gene track (your current version) -----------------

    gene_sel <- gene_track_plot(gene_name = gene_name, chrom = chrom, 
                                  bmap = bmap, bigStart = bigStart, bigEnd = bigEnd, midpoint = midpoint)
    gene_sel <- gene_sel %>%
      mutate(row = as.numeric(factor(gene)))  
    
    gene_plot <- gene_sel %>%
      group_by(gene, row, strand) %>%
      summarise(start_gene = min(start), stop_gene = max(stop), .groups = "drop") %>%
      mutate(label_x = ifelse(strand == "+", start_gene - 0.05*(bigEnd-bigStart)*1000,
                              stop_gene + 0.05*(bigEnd-bigStart)*1000),
             hjust_val = ifelse(strand == "+", 0, 1),
             highlight = (gene == gene_name))
    
    gene_sel <- gene_sel %>%
      mutate(highlight = (gene == gene_name))
    
    g_genes <- ggplot() +
      geom_segment(data = gene_plot,
                   aes(x = start_gene, xend = stop_gene, y = row, yend = row),
                   size = 0.1, color = "gray70") +
      geom_rect(data = gene_sel,
                aes(xmin = start, xmax = stop, ymin = row - 0.2, ymax = row + 0.2,
                    fill = highlight),
                color = "black",size = 0.1) +
      scale_fill_manual(values = c("TRUE" = "orange", "FALSE" = "gray80"), guide="none") +
      geom_text(data = gene_plot,
                aes(x = label_x, y = row, label = gene),
                size = 3) +
      scale_y_continuous(breaks = NULL) +
      theme_minimal(base_size = 10) +
      theme(panel.grid = element_blank())+
      labs(y = "Genes", fill = "") +
      coord_cartesian(xlim = c(bigStart*1000, bigEnd*1000))
    
    # ----------------- Epigenetic tracks -----------------
    safe_df <- function(beddata) {
      if (!is.null(beddata) && nrow(beddata) > 0) as.data.frame(beddata)
      else data.frame("peak_start" = numeric(0), "peak_end" = numeric(0))
    }
    
    atac_df  <- safe_df(atac)
    k27ac_df <- safe_df(k27ac)
    k4me3_df <- safe_df(k4me3)
    
    plot_chip <- function(df, label) {
      ggplot(df, aes(xmin = peak_start, xmax = peak_end, ymin = 0, ymax = 1)) +
        geom_rect(fill = "grey40") +
        labs(y = label, x = NULL) +
        coord_cartesian(xlim = c(bigStart*1000, bigEnd*1000), ylim = c(0,1)) +
        theme_minimal(base_size = 10) +
        theme(panel.grid = element_blank(), 
              panel.border = element_blank(),
              axis.text.x=element_blank(),
              axis.ticks.x=element_blank(),
             axis.text.y=element_blank(),
             axis.ticks.y=element_blank())
    }
    
    g_atac  <- plot_chip(atac_df,  "ATAC")
    g_k27ac <- plot_chip(k27ac_df, "H3K27ac")
    g_k4me3 <- plot_chip(k4me3_df, "H3K4me3")
    
    # ----------------- Final combined plot -----------------
    final_plot <- (g_contacts / g_atac / g_k27ac / g_k4me3 / g_genes) +
      plot_layout(heights = c(2,0.2, 0.2, 0.2, 0.5), guides = "collect")
    
    # ----------------- Save to PDF -----------------
    print(final_plot)
    dev.off()
  }
}
